#### Clean/ Preprocess Scraped Movie Data

In [1]:
import numpy as np
import pandas as pd

#Read in Piclkles and create Pandas Dataframe
def create_dataframe(start_year,stop_year):
    data=[]
    
    for yr in range(start_year,(stop_year+1)):
        pickle=pd.read_pickle("data/movie_data_raw_"+str(yr)+".pickle")
        data+=pickle
        
        print yr, len(pickle), len(data)
        
    df=pd.DataFrame(data)
    df=df[(df["release_year"]>=start_year) & (df["release_year"]<=stop_year)]
    
    return df
        
df=create_dataframe(2005,2015)

print df.columns
print df.shape

print df["release_year"].value_counts(dropna=False)

2005 553 553
2006 615 1168
2007 638 1806
2008 614 2420
2009 527 2947
2010 542 3489
2011 609 4098
2012 672 4770
2013 695 5465
2014 710 6175
2015 699 6874
Index([u'budget', u'genre', u'imdb_id', u'imdb_rating', u'metascore',
       u'oscar_noms', u'oscar_wins', u'plot', u'rating', u'release_date',
       u'release_year', u'runtime', u'title', u'tomato_meter',
       u'tomato_user_meter'],
      dtype='object')
(6797, 15)
2014    702
2015    690
2013    688
2012    665
2007    631
2006    608
2008    607
2011    602
2005    547
2010    536
2009    521
Name: release_year, dtype: int64


In [2]:
print df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6797 entries, 0 to 6873
Data columns (total 15 columns):
budget               1491 non-null float64
genre                6797 non-null object
imdb_id              3999 non-null object
imdb_rating          3999 non-null object
metascore            3999 non-null object
oscar_noms           6797 non-null int64
oscar_wins           6797 non-null int64
plot                 3999 non-null object
rating               6797 non-null object
release_date         6797 non-null datetime64[ns]
release_year         6797 non-null int64
runtime              6574 non-null float64
title                6797 non-null object
tomato_meter         3999 non-null object
tomato_user_meter    3999 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(9)
memory usage: 849.6+ KB
None


In [3]:
df.head(10)

,budget,genre,imdb_id,imdb_rating,metascore,oscar_noms,oscar_wins,plot,rating,release_date,release_year,runtime,title,tomato_meter,tomato_user_meter
0,26000000,Comedy,tt0405422,7.2,73,0,0,"Goaded by his buddies, a nerdy guy who's never...",R,2005-08-19,2005,116,The 40-Year-Old Virgin,85,84
1,62000000,Sci-Fi Action,tt0402022,5.5,36,0,0,Aeon Flux is a mysterious assassin working for...,PG-13,2005-12-02,2005,95,Aeon Flux,10,39
2,19000000,Horror,tt0384806,6.0,33,0,0,Newlyweds are terrorized by demonic forces aft...,R,2005-04-15,2005,89,The Amityville Horror,23,52
3,32000000,Family Comedy,tt0368578,4.5,27,0,0,Nick persons is a rich but selfish player who ...,PG,2005-01-21,2005,94,Are We There Yet?,11,45
4,35000000,Sports Comedy,tt0408524,5.8,65,0,0,A grizzled little league coach tries to turn h...,PG-13,2005-07-22,2005,111,Bad News Bears,N/A,0
5,150000000,Action / Adventure,tt0372784,8.3,70,1,0,"After training with his mentor, Batman begins ...",PG-13,2005-06-15,2005,140,Batman Begins,85,94
6,NaN,Comedy,tt0388500,5.5,53,0,0,A determined hairstylist (Queen Latifah) compe...,PG-13,2005-03-30,2005,105,Beauty Shop,37,62
7,53000000,Crime Comedy,tt0377471,5.7,37,0,0,"Disenchanted with the movie industry, Chili Pa...",PG-13,2005-03-04,2005,104,Be Cool,30,42
8,85000000,Romantic Comedy,tt0374536,4.8,34,0,0,Thinking he can overshadow an unknown actress ...,PG-13,2005-06-24,2005,90,Bewitched,25,28
9,20000000,Horror,tt0357507,4.1,32,0,0,A young man tries to deal with the childhood t...,PG-13,2005-02-04,2005,86,Boogeyman,13,21


In [4]:
#CLEAN GENRE - Pool Similar Genres into More General Categories
#Output List of Genres to CSV File to group (by hand), then read back in coded genres and join to dataframe

#df["genre"].value_counts().to_csv("data/genre_counts.csv")
genre_coded=pd.read_csv("data/genre_counts_coded.csv")
df=pd.merge(df,genre_coded,on="genre")

print df["genre_group"].value_counts(dropna=False)
print df.columns
print df.shape

Drama                       1394
Foreign                     1241
Comedy                      1174
Documentary                 1121
Horror/ Thriller/ Sci-Fi     799
Action/ Adventure            437
Other                        328
Family/ Animation            284
NaN                           19
Name: genre_group, dtype: int64
Index([u'budget', u'genre', u'imdb_id', u'imdb_rating', u'metascore',
       u'oscar_noms', u'oscar_wins', u'plot', u'rating', u'release_date',
       u'release_year', u'runtime', u'title', u'tomato_meter',
       u'tomato_user_meter', u'counts', u'genre_group', u'oscar_bait_genre'],
      dtype='object')
(6797, 18)


In [5]:
#CLEAN MPAA RATING - Pool G/PG and R/NC-17
print df["rating"].value_counts(dropna=False)

df.ix[df["rating"]=="Unknown", "rating"]=np.nan
df.ix[df["rating"].isin(["G","PG"]), "rating"]="G/ PG"
df.ix[df["rating"].isin(["R","NC-17"]), "rating"]="R/ NC17"

print df["rating"].value_counts(dropna=False)

Unrated    2649
R          2094
PG-13      1360
PG          576
G           106
NC-17        10
Unknown       2
Name: rating, dtype: int64
Unrated    2649
R/ NC17    2104
PG-13      1360
G/ PG       682
NaN           2
Name: rating, dtype: int64


In [6]:
#CLEAN User Ratings, etc.
def clean_ratings(var):
    print df[var].value_counts(dropna=False).head(5)
    
    df[var]=pd.to_numeric(df[var], errors="coerce")
    
    print df[var].value_counts(dropna=False).head(5)

clean_ratings("imdb_rating")
clean_ratings("metascore")
clean_ratings("tomato_meter")
clean_ratings("tomato_user_meter")

NaN    2798
6.7     159
6.5     157
7.0     154
7.2     151
Name: imdb_rating, dtype: int64
NaN     2926
 6.7     159
 6.5     157
 7.0     154
 7.2     151
Name: imdb_rating, dtype: int64
NaN    2798
N/A    1039
55       72
53       66
57       66
Name: metascore, dtype: int64
NaN    3837
 55      72
 53      66
 57      66
 58      66
Name: metascore, dtype: int64
NaN    2798
N/A     869
83       61
89       59
71       53
Name: tomato_meter, dtype: int64
NaN    3667
 83      61
 89      59
 71      53
 80      52
Name: tomato_meter, dtype: int64
NaN    2798
N/A     377
50       85
75       80
71       75
Name: tomato_user_meter, dtype: int64
NaN    3175
 50      85
 75      80
 71      75
 67      73
Name: tomato_user_meter, dtype: int64


In [7]:
#Create A Variable Showing Quarter of Year in which film was produced
df["release_qtr"]=0
df.ix[df["release_date"].dt.month.isin([1,2,3]), "release_qtr"]=1
df.ix[df["release_date"].dt.month.isin([4,5,6]), "release_qtr"]=2
df.ix[df["release_date"].dt.month.isin([7,8,9]), "release_qtr"]=3
df.ix[df["release_date"].dt.month.isin([10,11,12]), "release_qtr"]=4

print df["release_qtr"].value_counts(dropna=False)

3    1822
2    1739
4    1722
1    1514
Name: release_qtr, dtype: int64


In [8]:
#Clean Up Production Budget
df["budget"]=df.budget/1000 #normalize budget in thousands 

df["budget"].describe()

print df["budget"].quantile(q=0.01)
print df["budget"].quantile(q=0.05)
print df["budget"].quantile(q=0.95)
print df["budget"].quantile(q=0.99)

#clean outliers
df.ix[df.budget<1800, "budget"]=np.nan
df.ix[df.budget>142000, "budget"]=np.nan

print df["budget"].describe()

80.4585
1792.0
141972.0
196345.0
count      1338.000000
mean      38100.590994
std       33426.610041
min        1884.000000
25%       13791.850000
50%       27210.000000
75%       52945.000000
max      141484.000000
Name: budget, dtype: float64


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6797 entries, 0 to 6796
Data columns (total 19 columns):
budget               1338 non-null float64
genre                6797 non-null object
imdb_id              3999 non-null object
imdb_rating          3871 non-null float64
metascore            2960 non-null float64
oscar_noms           6797 non-null int64
oscar_wins           6797 non-null int64
plot                 3999 non-null object
rating               6795 non-null object
release_date         6797 non-null datetime64[ns]
release_year         6797 non-null int64
runtime              6574 non-null float64
title                6797 non-null object
tomato_meter         3130 non-null float64
tomato_user_meter    3622 non-null float64
counts               6797 non-null int64
genre_group          6778 non-null object
oscar_bait_genre     6797 non-null int64
release_qtr          6797 non-null int64
dtypes: datetime64[ns](1), float64(6), int64(6), object(6)
memory usage: 1.0+ MB


In [10]:
#Split Cleaned Data into Train and Test & Pickle for Exploration and Modeling
df.to_pickle("data/movie_data_clean.pickle")